In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [3]:
path = r"Data\spambase.csv"

In [35]:
data = pd.read_csv(path)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 58 columns):
0         4600 non-null float64
0.64      4600 non-null float64
0.64.1    4600 non-null float64
0.1       4600 non-null float64
0.32      4600 non-null float64
0.2       4600 non-null float64
0.3       4600 non-null float64
0.4       4600 non-null float64
0.5       4600 non-null float64
0.6       4600 non-null float64
0.7       4600 non-null float64
0.64.2    4600 non-null float64
0.8       4600 non-null float64
0.9       4600 non-null float64
0.10      4600 non-null float64
0.32.1    4600 non-null float64
0.11      4600 non-null float64
1.29      4600 non-null float64
1.93      4600 non-null float64
0.12      4600 non-null float64
0.96      4600 non-null float64
0.13      4600 non-null float64
0.14      4600 non-null float64
0.15      4600 non-null float64
0.16      4600 non-null float64
0.17      4600 non-null float64
0.18      4600 non-null float64
0.19      4600 non-null float

In [6]:
data.describe()

,0,0.64,0.64.1,0.1,0.32,0.2,0.3,0.4,0.5,0.6,...,0.40,0.41,0.42,0.778,0.43,0.44,3.756,61,278,1
count,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,...,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000
mean,0.104576,0.212922,0.280578,0.065439,0.312222,0.095922,0.114233,0.105317,0.090087,0.239465,...,0.038583,0.139061,0.016980,0.268960,0.075827,0.044248,5.191827,52.170870,283.290435,0.393913
std,0.305387,1.290700,0.504170,1.395303,0.672586,0.273850,0.391480,0.401112,0.278643,0.644816,...,0.243497,0.270377,0.109406,0.815726,0.245906,0.429388,31.732891,194.912453,606.413764,0.488669
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.275500,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.382500,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.314250,0.052000,0.000000,3.705250,43.000000,265.250000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [7]:
data.head()

,0,0.64,0.64.1,0.1,0.32,0.2,0.3,0.4,0.5,0.6,...,0.40,0.41,0.42,0.778,0.43,0.44,3.756,61,278,1
0,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
1,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
2,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.00,0.223,0.0,0.000,0.000,0.000,3.000,15,54,1


In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns="1"), data["1"], test_size=0.3, random_state=6)
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
accuracy_score = lr_model.score(X_test, y_test)
accuracy_score

0.9260869565217391

In [10]:
y_pred = lr_model.predict(X_test)

In [11]:
cr = classification_report(y_test, y_pred)
cr

'              precision    recall  f1-score   support\n\n           0       0.93      0.95      0.94       845\n           1       0.92      0.89      0.90       535\n\n    accuracy                           0.93      1380\n   macro avg       0.92      0.92      0.92      1380\nweighted avg       0.93      0.93      0.93      1380\n'

In [12]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[804,  41],
       [ 61, 474]], dtype=int64)

In [25]:
df1 = data
df1 = df1.corr().unstack()
df1 = df1[(abs(df1)>0.75) & (abs(df1)!=1)]

In [27]:
df1

0.23  0.25    0.996066
      0.31    0.848020
0.25  0.23    0.996066
      0.31    0.845358
0.31  0.23    0.848020
      0.25    0.845358
dtype: float64

In [36]:
df1 = data.drop(columns=["0.25", "0.31"])

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df1.drop(columns="1"), df1["1"], test_size=0.3, random_state=6)
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
accuracy_score = lr_model.score(X_test, y_test)
accuracy_score

0.927536231884058

In [39]:
y_pred = lr_model.predict(X_test)
cr = classification_report(y_test, y_pred)
cr

'              precision    recall  f1-score   support\n\n           0       0.93      0.95      0.94       845\n           1       0.92      0.89      0.90       535\n\n    accuracy                           0.93      1380\n   macro avg       0.93      0.92      0.92      1380\nweighted avg       0.93      0.93      0.93      1380\n'

In [40]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[806,  39],
       [ 61, 474]], dtype=int64)

In [44]:
test = SelectKBest(score_func=chi2, k='all')

In [45]:
X_train = test.fit_transform(X_train, y_train)
X_test = test.transform(X_test)

In [50]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [52]:
score = lr.score(X_test, y_test)
score

0.927536231884058

In [53]:
cm = confusion_matrix(y_test, lr.predict(X_test))
cm

array([[806,  39],
       [ 61, 474]], dtype=int64)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(df1.drop(columns="1"), df1["1"], test_size=0.3, random_state=6)
pca = PCA(n_components=35, random_state=0)
X_train = pca.fit_transform(X_train, y_train)
X_test = pca.transform(X_test)
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_perd = lr.predict(X_test)
score = lr.score(X_test, y_test)
cm = confusion_matrix(y_test, y_pred)
score, cm

(0.9173913043478261, array([[806,  39],
        [ 61, 474]], dtype=int64))